(case-warfarin)=

# Warfarin PopPk

In this document, the Warfarin dataset is used to demonstrate the workflow of exploratory data analysis, model fitting, and simulation.

## Exploratory data analysis

In [1]:
from mas.model import *
import pandas as pd

model_data = ModelData(pd.read_csv(
    'https://clinpharmacol.fmhs.auckland.ac.nz/docs/warfarin.csv', 
    na_values=["."], 
    comment="#", 
    header=None, 
    names=["ID", "time", "wt", "age", "sex", "amt", "rate", "dvid", "dv", "mdv"]
))
model_data = model_data.drop(columns=["rate"])
model_data = model_data[model_data["dvid"] != 2].reset_index(drop=True)
model_data.head(20)

    ID   time    wt  age  sex    amt  dvid    dv  mdv
0    0    0.0  66.7   50    1  100.0     0   NaN    1
1    0    0.5  66.7   50    1    NaN     1   0.0    0
2    0    1.0  66.7   50    1    NaN     1   1.9    0
3    0    2.0  66.7   50    1    NaN     1   3.3    0
4    0    3.0  66.7   50    1    NaN     1   6.6    0
5    0    6.0  66.7   50    1    NaN     1   9.1    0
6    0    9.0  66.7   50    1    NaN     1  10.8    0
7    0   12.0  66.7   50    1    NaN     1   8.6    0
8    0   24.0  66.7   50    1    NaN     1   5.6    0
9    0   36.0  66.7   50    1    NaN     1   4.0    0
10   0   48.0  66.7   50    1    NaN     1   2.7    0
11   0   72.0  66.7   50    1    NaN     1   0.8    0
12   0   96.0  66.7   50    1    NaN     1   NaN    1
13   0  120.0  66.7   50    1    NaN     1   NaN    1
14   1    0.0  66.7   50    1  100.0     0   NaN    1
15   1   24.0  66.7   50    1    NaN     1   9.2    0
16   1   36.0  66.7   50    1    NaN     1   8.5    0
17   1   48.0  66.7   50    

The concentration-time profile is shown below.

In [2]:
model_data.explore_contour_plot()

Additionally, there is an option to display the plot with a logarithmic scale on the y-axis.

In [3]:
model_data.explore_contour_plot(y_scale='log')

## Model building

In [4]:
class Warfarin(pk.EvOneCmtLinear):

    def __init__(self) -> None:
        super().__init__()
        self.tv_cl = theta(0.134, bounds=(0, None))
        self.tv_v = theta(7.81, bounds=(0, None))
        self.tv_ka = theta(0.571, bounds=(0, None))
        self.tv_alag = theta(0.823, bounds=(0, None))

        self.eta_cl = omega(0.049)
        self.eta_v = omega(0.0802)
        self.eta_ka = omega(0.047)
        self.eta_alag = omega(0.156)

        self.eps_prop = sigma(0.0104)
        self.eps_add = sigma(0.554)

    def pred(self) -> Expression:
        cl = self.tv_cl * exp(self.eta_cl)
        v = self.tv_v * exp(self.eta_v)
        ka = self.tv_ka * exp(self.eta_ka)
        alag = self.tv_alag * exp(self.eta_alag)

        computed = self.trans2(cl=cl, v=v, ka=ka, alag1=alag)
        ipred = computed.F
        
        y = ipred * (1 + self.eps_prop) + self.eps_add

        return y


pop_model = PopModel(Warfarin, data=model_data)

➡️ Since build cache already exists, skip compile


We fit the model using the FOCEi estimation method and specify using the BOBYQA algorithm to minimize the objective function. The standard errors are computed after the estimation of parameters is completed.

In [5]:
fit_result = pop_model.fit(FOCEi(minimize_method='bobyqa', print_type='progress'))

✈️ First Order Conditional Estimation

* maxiter = 2000
* xtol = 0.000001
* ftol = 0.000001
* lower_bounds = 
* upper_bounds = 
* log_level = 1
* print = 1
* print_type = 1
Using Bobyqa...
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 50
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 100
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 150
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 200
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 250
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 300
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 350
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 400
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || =====

In [6]:
fit_result.compute_se()

array([[ 0.00659271,  0.38859566, -0.08432029, -0.10437474,  0.33376306,
        -0.09358475,  0.47255923,  0.07946763,  0.27140031,  0.17313529],
       [ 0.38859566,  0.31254792, -0.01857391, -0.09650876, -0.26137131,
         0.11996913,  0.08797729,  0.07367183, -0.11039761, -0.47831444],
       [-0.08432029, -0.01857391,  0.23756905,  0.25042897, -0.29916443,
         0.03220941,  0.01948626, -0.63004992,  0.082401  , -0.00118228],
       [-0.10437474, -0.09650876,  0.25042897,  0.03709139, -0.44457168,
         0.13172403, -0.29394196, -0.30149834,  0.12741986,  0.19135797],
       [ 0.33376306, -0.26137131, -0.29916443, -0.44457168,  0.02202855,
         0.26501768,  0.4144288 ,  0.33760049,  0.08719855,  0.21897336],
       [-0.09358475,  0.11996913,  0.03220941,  0.13172403,  0.26501768,
         0.01207818, -0.26524773,  0.01695377, -0.20972812,  0.03946084],
       [ 0.47255923,  0.08797729,  0.01948626, -0.29394196,  0.4144288 ,
        -0.26524773,  0.17019961,  0.37049334

Below is a summary of the estimation result.

In [7]:
print(fit_result)

================= Summary =================

*** Fit Summary ***
| Final Objective Value   | Converged   | Total CPU Elapsed Time   | Total Iterations   |
|-------------------------|-------------|--------------------------|--------------------|
| 216.802                 | True ✅     | 0:00:02.73               | 521                |

*** Parameter Estimation ***
| Parameter       | Estimate   | RSE(%)   | Shrinkage(%)   |
|-----------------|------------|----------|----------------|
| **Theta**       |            |          |                |
| tv_cl           | 0.132      | 5.0      | -              |
| tv_v            | 7.969      | 3.9      | -              |
| tv_ka           | 1.407      | 16.9     | -              |
| tv_alag         | 0.882      | 4.2      | -              |
| **Omega[SD]**   |            |          |                |
| Omega(eta_cl)   | 0.291      | 13.0     | 1.5            |
| Omega(eta_v)    | 0.218      | 12.7     | 5.3            |
| Omega(eta_ka)   | 0.795 

The diagnostic plots are presented interactively, allowing you to zoom in and examine the details. By hovering your mouse pointer over the scatter plots, you can easily identify the outliers by their corresponding ID.

In [8]:
fit_result.plot_goodness_of_fit()

In [9]:
fit_result.plot_individual_fit(cols=3)

## VPC

In [15]:
from mas.libs.masmod.api.estimation.vpc import vpc

vpc_result = vpc(fit_result, samples=500)
vpc_result

Auto-bin runing.
Ending bin search early, Nmax  10
Auto-bin finally use 10 bins.



## Simulation

In [11]:
import numpy as np

# specify a simulation scenario involving a single dose of 120mg,
# and an observation time ranging from 0 to 120 hours, 
# divided into 200 evenly spaced time intervals (N=100)
simu_events = et(time=0, amt=120) + \
              et(time=np.linspace(0, 120, 200)) + \
              et(id=np.arange(100))

We simulate based on the model estimation `fit_result` and the simulation events `simu_events`.

In [12]:
simu_result = fit_result.simulate(data=simu_events)

In [13]:
simu_result.plot_individual(y_axis='ipred')

In [14]:
simu_result.plot_interval(y_axis='ipred')